In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

In [6]:
import zipfile
import pandas as pd
import io

# Path to ZIP file
zip_path = 'C:/Users/Asus/AppData/Local/Temp/d8a7ca88-755a-41b9-94d1-c1ea4c9477b3_dataset.zip.7b3/AirQuality.xlsx.zip'

# Open the ZIP and read the Excel file inside
with zipfile.ZipFile(zip_path, 'r') as z:
    # List file names to confirm the correct Excel file name
    print("Files in ZIP:", z.namelist())
    with z.open('AirQualityUCI.xlsx') as f:  # Make sure the name matches exactly
        df = pd.read_excel(f)

# Continue with your preprocessing
df = df.iloc[:, :-2]
df.replace(-200, pd.NA, inplace=True)
df.columns = df.columns.str.strip()


Files in ZIP: ['AirQualityUCI.xlsx']


In [8]:
import zipfile
import pandas as pd

# Path to ZIP file
zip_path = 'C:/Users/Asus/AppData/Local/Temp/d8a7ca88-755a-41b9-94d1-c1ea4c9477b3_dataset.zip.7b3/AirQuality.xlsx.zip'

try:
    # Open the ZIP and read the Excel file
    with zipfile.ZipFile(zip_path, 'r') as z:
        with z.open('AirQualityUCI.xlsx') as f:
            df = pd.read_excel(f, engine='openpyxl')

    # Preprocessing
    df = df.iloc[:, :-2]  # Remove last two columns
    df.replace(-200, pd.NA, inplace=True)  # Replace -200 with NA
    df.columns = df.columns.str.strip()  # Clean column names

    # Display basic info
    print("DataFrame Info:")
    print(df.info())
    print("\nFirst few rows:")
    print(df.head())

except Exception as e:
    print(f"Error: {e}")



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           9357 non-null   datetime64[ns]
 1   Time           9357 non-null   object        
 2   CO(GT)         7674 non-null   object        
 3   PT08.S1(CO)    8991 non-null   object        
 4   NMHC(GT)       914 non-null    object        
 5   C6H6(GT)       8991 non-null   object        
 6   PT08.S2(NMHC)  8991 non-null   object        
 7   NOx(GT)        7718 non-null   object        
 8   PT08.S3(NOx)   8991 non-null   object        
 9   NO2(GT)        7715 non-null   object        
 10  PT08.S4(NO2)   8991 non-null   object        
 11  PT08.S5(O3)    8991 non-null   object        
 12  T              8991 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 950.4+ KB
None

First few rows:
        Date      Time CO(GT

In [10]:
# Feature Engineering
# 1. Lagged features (previous 1, 3, 6 hours)
for col in df.columns:
    df[f'{col}_lag1'] = df[col].shift(1)
    df[f'{col}_lag3'] = df[col].shift(3)
    df[f'{col}_lag6'] = df[col].shift(6)

In [13]:
# 2. Rolling mean (window of 3 hours)
for col in df.columns:
    if 'lag' not in col and df[col].dtype in ['float64', 'int64']:  # Only numeric columns
        df[f'{col}_roll_mean3'] = df[col].rolling(window=3).mean()

In [14]:
# Create rolling mean features (window of 3 hours)
for col in df.columns:
    if 'lag' not in col and df[col].dtype in ['float64', 'int64']:  # Exclude lags and non-numeric
        df[f'{col}_roll_mean3'] = df[col].rolling(window=3).mean()

In [15]:
# Drop rows with NaN values from lagging/rolling
df = df.dropna()

In [16]:
# 3. Define Targets
targets = ['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)', 
           'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 
           'T', 'RH', 'AH']

In [17]:
# 4. Train-Test Split (Last 10% for RMSE evaluation)
train_size = int(len(df) * 0.9)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

In [24]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/150.0 MB 7.5 MB/s eta 0:00:20
    --------------------------------------- 3.1/150.0 MB 7.7 MB/s eta 0:00:20
   - -------------------------------------- 4.7/150.0 MB 7.7 MB/s eta 0:00:19
   - -------------------------------------- 6.6/150.0 MB 7.7 MB/s eta 0:00:19
   -- ------------------------------------- 8.1/150.0 MB 7.9 MB/s eta 0:00:19
   -- ------------------------------------- 9.4/150.0 MB 7.6 MB/s eta 0:00:19
   -- ------------------------------------- 10.7/150.0 MB 7.3 MB/s eta 0:00:20
   --- ------------------------------------ 11.8/150.0 MB 7.2 MB/s eta 0:00:20
   --- ------------------------------------ 13.4/150.0 MB 7.1 MB/s eta 0:00:20
   --- ------------------------------------ 14.7/150.0 MB 7.0 MB/s eta 0:00:20
   ---- ----------------------------------- 16.0/150.0 MB 6.9 MB/s eta 0:00:20
   ---- ----------------------------------- 17.0/150.0 MB 6.8 MB/s

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error

# --- Dummy data creation for demonstration ---
# Replace this block with your actual forecasting code that generates forecast_df

# Create a datetime index starting one hour after the last known time
last_timestamp = pd.Timestamp('2023-01-01 00:00:00')  # Replace with your last datetime from df.index
forecast_index = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1), periods=48, freq='h')  # Changed 'H' to 'h'

# Create dummy forecast data for pollutants (example with CO(GT))
forecast_data = {
    'CO(GT)': np.random.rand(48) * 5  # Replace with your forecast values
}
forecast_df = pd.DataFrame(data=forecast_data, index=forecast_index)

# Example targets and feature_importances dictionaries (replace with your actual data)
targets = ['CO(GT)']
feature_importances = {
    'RandomForest_CO(GT)': pd.Series(np.random.rand(5), index=['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5']),
    'XGBoost_CO(GT)': pd.Series(np.random.rand(5), index=['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5'])
}
residuals = {
    'RandomForest_CO(GT)': np.random.randn(100),  # Replace with your actual residuals
    'XGBoost_CO(GT)': np.random.randn(100)
}

# --- Save forecast ---
forecast_df.to_csv('forecast_48_hours.csv')

# --- Additional Analysis ---
if 'CO(GT)' in targets and 'RandomForest_CO(GT)' in feature_importances:
    plt.figure(figsize=(10, 6))
    feature_importances['RandomForest_CO(GT)'].sort_values().plot(kind='barh')
    plt.title('Feature Importance for CO(GT) - RandomForest')
    plt.savefig('feature_importance_co_rf.png')
    plt.close()

if 'CO(GT)' in targets and 'XGBoost_CO(GT)' in feature_importances:
    plt.figure(figsize=(10, 6))
    feature_importances['XGBoost_CO(GT)'].sort_values().plot(kind='barh')
    plt.title('Feature Importance for CO(GT) - XGBoost')
    plt.savefig('feature_importance_co_xgb.png')
    plt.close()

if 'CO(GT)' in targets and 'RandomForest_CO(GT)' in residuals:
    plt.figure(figsize=(10, 4))
    plot_acf(residuals['RandomForest_CO(GT)'], lags=24, title='Autocorrelation of Residuals for CO(GT) - RandomForest')
    plt.savefig('residuals_acf_co_rf.png')
    plt.close()

if 'CO(GT)' in targets and 'XGBoost_CO(GT)' in residuals:
    plt.figure(figsize=(10, 4))
    plot_acf(residuals['XGBoost_CO(GT)'], lags=24, title='Autocorrelation of Residuals for CO(GT) - XGBoost')
    plt.savefig('residuals_acf_co_xgb.png')
    plt.close()

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>